In [ ]:
import pandas as pd
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# Load dataset
# df = pd.read_csv('drive/My Drive/Colab Notebooks/Base.csv')
df = pd.read_csv('E:/python2222/data/Base.csv')

# Specify the label column (the label column name in Base.csv is fraud_bool)
label_col = "fraud_bool"
if label_col not in df.columns:
    raise KeyError(f"Label column '{label_col}' not found. Please verify the label column name in the CSV and modify label_col accordingly.")

# Construct X, y (these variables will be used by the model later)
y = df[label_col].astype(int)
X = df.drop(columns=[label_col])

print("\nLabel distribution:")
print(y.value_counts(dropna=False))
print("Positive rate = {:.6f}".format(y.mean()))
print("\nFirst 5 rows:")
display(df.head())
# For convenience in subsequent steps, save basic information
print("\nNote: We will use variables X, y, df as the main data objects in the following steps.")


In [ ]:
# === : Automatically identify numeric/categorical/ID/time/high-cardinality columns (check output after running) ===
import pandas as pd
# 1) Data types and unique value statistics for each column
col_nunique = df.nunique(dropna=False).sort_values()
col_dtype = df.dtypes
summary_df = pd.DataFrame({
    "dtype": col_dtype,
    "nunique": col_nunique,
    "unique_ratio": col_nunique / len(df),
    "missing": df.isna().sum()
}).sort_values("nunique", ascending=False)
display(summary_df.head(50))  # Display top 50 columns with more unique values
# 2) Automatically separate numeric and categorical columns (for reference, manual verification needed)
num_cols_auto = df.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols_auto = df.select_dtypes(include=["object","category","bool"]).columns.tolist()
print("\nAutomatic detection (for reference only):")
print("Numeric columns (count={}, first 30 examples):".format(len(num_cols_auto)), num_cols_auto[:30])
print("Categorical columns (count={}, first 30 examples):".format(len(cat_cols_auto)), cat_cols_auto[:30])
# 3) Identify possible ID columns (high cardinality / unique_ratio close to 1 or very large nunique)
possible_id_cols = []
for c in df.columns:
    if summary_df.loc[c, "unique_ratio"] > 0.95:
        possible_id_cols.append((c, int(summary_df.loc[c, "nunique"])))
# Also consider column names explicitly containing 'id' as possible IDs
for c in df.columns:
    if "id" in c.lower() or c.lower().endswith("_id"):
        if c not in [x[0] for x in possible_id_cols]:
            possible_id_cols.append((c, int(summary_df.loc[c, "nunique"])))
print("\nSuggested ID-like columns to check (unique_ratio>0.95 or name contains 'id'):")
print(possible_id_cols[:50])
# 4) Identify possible time columns (column names containing date/time or successfully parsed as datetimes)
possible_time_cols = []
date_like_names = [c for c in df.columns if any(k in c.lower() for k in ("date","time","day","month","year","ts"))]
for c in date_like_names:
    # Try parsing the first 1000 non-null values and check the success rate
    sample = df[c].dropna().astype(str).head(1000)
    parsed = pd.to_datetime(sample, errors="coerce")
    success_ratio = parsed.notna().mean() if len(parsed)>0 else 0.0
    if success_ratio > 0.8:
        possible_time_cols.append((c, success_ratio))
    else:
        # Record but also note if success rate is low
        possible_time_cols.append((c, success_ratio))
print("\ndate/time-like columns (with parsing success rate):")
print(possible_time_cols)
# 5) High-cardinality categorical columns (categorical columns with large unique count, not friendly for OneHot)
high_card_cat = []
for c in cat_cols_auto:
    nun = int(summary_df.loc[c,"nunique"])
    if nun > 100:  # Threshold can be adjusted (100 is just an empirical value)
        high_card_cat.append((c, nun))
print("\nHigh-cardinality categorical columns (nunique>100):")
print(high_card_cat[:50])
# 6) Top few values for each categorical column (for confirmation)
print("\nExample values for categorical columns (showing up to 10 most frequent values per column):")
for c in cat_cols_auto[:50]:
    top = df[c].value_counts(dropna=False).head(10)
    print(f"== {c} (nunique={int(summary_df.loc[c,'nunique'])}) ==")
    print(top.to_dict())
    print()

In [ ]:
# Columns to exclude from features
exclude_cols = [ "prev_address_months_count", # Features with excessive missing samples that are difficult to impute
                 "month"]  # Timestamp

# === Organize num_cols/cat_cols, build ColumnTransformer, and perform stratified train/test split ===

num_cols = ['income', 'name_email_similarity', 'current_address_months_count', 'customer_age',
            'days_since_request', 'intended_balcon_amount', 'zip_count_4w', 'velocity_6h',
            'velocity_24h', 'velocity_4w', 'bank_branch_count_8w', 'date_of_birth_distinct_emails_4w',
            'credit_risk_score', 'bank_months_count', 'proposed_credit_limit', 'session_length_in_minutes']

cat_cols = ['payment_type', 'employment_status', 'email_is_free', 'housing_status', 'phone_home_valid',
            'phone_mobile_valid', 'has_other_cards', 'foreign_request', 'source', 'device_os',
            'keep_alive_session', 'device_distinct_emails_8w', 'device_fraud_count']

print("Final num_cols count:", len(num_cols))
print("Final cat_cols count:", len(cat_cols))

In [ ]:
# Train/test split (stratified split with stratify=y to prevent positive class distribution imbalance)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import sklearn

X = df.drop(columns=[label_col])
y = df[label_col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

print("Dataset split completed. Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("Train positives:", int(y_train.sum()), " Test positives:", int(y_test.sum()))

# Numeric features: median imputation + standardization
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical features: constant imputation + OneHot (automatically select parameters based on sklearn version)
if sklearn.__version__ >= "1.2":
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="__missing__")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
    ])
else:  # Compatible with older versions
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="__missing__")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=True))
    ])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, num_cols),
    ("cat", categorical_transformer, cat_cols)
], remainder="drop", sparse_threshold=0.3)

# Fit preprocessor (only fit on training set to avoid data leakage), and transform train/test sets for inspection
preprocessor.fit(X_train)

X_train_t = preprocessor.transform(X_train)
X_test_t = preprocessor.transform(X_test)
print("Transformed types:", type(X_train_t), type(X_test_t))
print("Transformed shapes (train/test):", X_train_t.shape, X_test_t.shape)

# Simply extract feature names after preprocessing (for plotting feature importance later)
def get_feature_names_from_preprocessor(preproc):
    feature_names = []
    for name, trans, cols in preproc.transformers_:
        if name == "num":
            feature_names.extend(cols)
        elif name == "cat":
            ohe = trans.named_steps["onehot"]
            try:
                ohe_names = ohe.get_feature_names_out(cols)
            except Exception:
                # Compatible with older sklearn versions
                ohe_names = []
                for i, col in enumerate(cols):
                    cats = ohe.categories_[i]
                    ohe_names.extend([f"{col}__{str(val)}" for val in cats])
            feature_names.extend(list(ohe_names))
    return feature_names

feat_names = get_feature_names_from_preprocessor(preprocessor)
print("Number of post-transform features (approx):", len(feat_names))
print("First 30 feature names:", feat_names[:30])

In [ ]:
# === 2025.10.17
import os, inspect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import StratifiedShuffleSplit

# Compatibility wrapper
def _get_feat_names_compat(preproc, X_columns):
    # 1) If your function exists and has input_features parameter
    try:
        if 'get_feature_names_from_preprocessor' in globals():
            sig = inspect.signature(get_feature_names_from_preprocessor)
            if 'input_features' in sig.parameters:
                return get_feature_names_from_preprocessor(preproc, input_features=X_columns)
    except Exception:
        pass

    # 2) If your function exists but without input_features parameter
    try:
        if 'get_feature_names_from_preprocessor' in globals():
            # Call your original version: def get_feature_names_from_preprocessor(preproc):
            return get_feature_names_from_preprocessor(preproc)
    except Exception:
        pass

    # 3) Fallback: manually construct feature names
    feature_names = []
    try:
        for name, trans, cols in preproc.transformers_:
            if name == "num":
                # Add numeric column names directly
                feature_names.extend(list(cols))
            elif name == "cat":
                # Expand categorical OneHot
                ohe = trans.named_steps.get("onehot", None)
                if ohe is not None:
                    try:
                        ohe_names = ohe.get_feature_names_out(cols)
                    except Exception:
                        # Compatible with older sklearn versions
                        ohe_names = []
                        for i, col in enumerate(cols):
                            cats = ohe.categories_[i]
                            ohe_names.extend([f"{col}__{str(val)}" for val in cats])
                    feature_names.extend(list(ohe_names))
                else:
                    # If no onehot step, append original columns directly
                    feature_names.extend(list(cols))
        return feature_names
    except Exception as e:
        print("Fallback building feature names failed:", e)
        # Last resort: fall back to original columns
        return list(X_columns)

def _get_preproc_and_names_from_pipeline(pipeline, X_columns):
    if hasattr(pipeline, 'named_steps') and 'preproc' in pipeline.named_steps:
        preproc = pipeline.named_steps['preproc']
    else:
        preproc = preprocessor   # Fallback
    feat_names = _get_feat_names_compat(preproc, X_columns)
    return preproc, feat_names

# === Plot: Top20 horizontal bar ===
def _plot_top_bar(importance_df, model_tag="Model", top_n=20, figsize=(10,7)):
    df = importance_df.head(top_n).sort_values('importance', ascending=True)
    plt.figure(figsize=figsize)
    sns.set_style("whitegrid")
    ax = sns.barplot(x='importance', y='feature', data=df, orient='h')
    plt.title(f"{model_tag} - Top {top_n} Feature Importance")
    plt.xlabel("Importance"); plt.ylabel("Feature")
    for i,(v,f) in enumerate(zip(df['importance'], df['feature'])):
        ax.text(v + 1e-6, i, f"{v:.4f}", va='center')
    plt.tight_layout(); plt.show()

# === Plot: Cumulative curve ===
def _plot_cumulative(importance_df, model_tag="Model", top_n=50, figsize=(8,4)):
    df = importance_df.copy().head(top_n).reset_index(drop=True)
    total = importance_df['importance'].sum() or 1.0
    cum_pct = df['importance'].cumsum()/total*100
    plt.figure(figsize=figsize)
    plt.plot(range(1, len(cum_pct)+1), cum_pct, marker='o')
    plt.xlabel("Number of features included"); plt.ylabel("Cumulative importance (%)")
    plt.title(f"{model_tag} cumulative importance (%)")
    plt.grid(True); plt.axhline(80, color='gray', linestyle='--', linewidth=1)
    plt.tight_layout(); plt.show()

# === Plot: Permutation uncertainty (violin & box) ===
def _plot_perm_uncertainty(perm_dict, model_tag="Model", top_n=20, figsize=(10,6)):
    if perm_dict is None: return
    all_imps = perm_dict['all']; means = perm_dict['means']; feats = list(perm_dict['feature_names'])
    order = np.argsort(means)[::-1][:top_n]
    sel = [feats[i] for i in order]
    rows=[]
    for i in order:
        for v in all_imps[i]:
            rows.append({'feature': feats[i], 'importance': v})
    long_df = pd.DataFrame(rows)
    plt.figure(figsize=figsize); sns.violinplot(x='importance', y='feature', data=long_df, order=sel, scale='width')
    plt.title(f"{model_tag} - Permutation distributions (violin)"); plt.tight_layout(); plt.show()
    plt.figure(figsize=figsize); sns.boxplot(x='importance', y='feature', data=long_df, order=sel)
    plt.title(f"{model_tag} - Permutation distributions (box)"); plt.tight_layout(); plt.show()

# === Core: Unified computation + plotting + export ===
def run_importance_all_models(pipeline, X, y, model_tag="Model",
                              top_n=20,
                              prefer='auto',         # 'auto'|'always_perm'|'no_perm'
                              n_repeats=10,
                              max_subsample=3000,    # Maximum subsample size for permutation
                              max_samples_per_repeat=2000,
                              n_jobs=1,              # Recommend 1 for Windows
                              scoring='f1_weighted',
                              out_dir="./importance_outputs"):
    """
    prefer:
      - 'auto'       : Prefer coef_/feature_importances_, otherwise use permutation
      - 'always_perm': Force permutation (for SVM rbf, etc.)
      - 'no_perm'    : Only use direct importance; skip if not available
    """
    os.makedirs(out_dir, exist_ok=True)

    # Get estimator & preproc & expanded names
    if hasattr(pipeline, 'named_steps'):
        est = pipeline.named_steps[list(pipeline.named_steps.keys())[-1]]
    else:
        est = pipeline
    preproc, feat_names = _get_preproc_and_names_from_pipeline(pipeline, X.columns)

    # ========== Case A: Has direct importance ==========
    if prefer in ('auto','no_perm') and hasattr(est, 'feature_importances_'):
        imps = est.feature_importances_
        imp_df = pd.DataFrame({'feature': feat_names, 'importance': imps}).sort_values('importance', ascending=False).reset_index(drop=True)
        imp_df.to_csv(os.path.join(out_dir, f"{model_tag}_feature_importances.csv"), index=False, encoding='utf-8-sig')
        _plot_top_bar(imp_df, model_tag, top_n); _plot_cumulative(imp_df, model_tag, max(50, top_n))
        return imp_df, None

    if prefer in ('auto','no_perm') and hasattr(est, 'coef_'):
        coefs = est.coef_
        imps = np.mean(np.abs(coefs), axis=0) if coefs.ndim>1 else np.abs(coefs).ravel()
        imp_df = pd.DataFrame({'feature': feat_names, 'importance': imps}).sort_values('importance', ascending=False).reset_index(drop=True)
        imp_df.to_csv(os.path.join(out_dir, f"{model_tag}_coef_importances.csv"), index=False, encoding='utf-8-sig')
        _plot_top_bar(imp_df, model_tag, top_n); _plot_cumulative(imp_df, model_tag, max(50, top_n))
        return imp_df, None

    if prefer == 'no_perm':
        print(f"[{model_tag}] no direct importances; permutation disabled -> skipped.")
        return None, None

    # ========== Case B: Do permutation (transform to expanded X, then permute on subsample) ==========
    # Transform once
    X_trans_full = preproc.transform(X)
    n_full = X_trans_full.shape[0]
    # Subsample (stratified, for speed)
    n_take = min(max_subsample, n_full)
    sss = StratifiedShuffleSplit(n_splits=1, train_size=n_take, random_state=42)
    idx = next(sss.split(np.zeros(n_full), y))[0]
    X_trans = X_trans_full[idx] if hasattr(X_trans_full, "tocsr") else X_trans_full[idx,:]
    y_sub = np.asarray(y)[idx]

    # Align y dtype
    y_arr = y_sub
    if hasattr(est, 'classes_'):
        cls = est.classes_
        try:
            y_arr = y_arr.astype(getattr(cls, 'dtype', y_arr.dtype), copy=False)
        except Exception:
            y_arr = y_arr.astype(str)

    # scorer & permutation
    scorer = make_scorer(f1_score, average=scoring.split('_')[-1]) if scoring!='accuracy' else 'accuracy'
    try:
        res = permutation_importance(est, X_trans, y_arr, n_repeats=n_repeats,
                                     random_state=42, n_jobs=n_jobs, scoring=scorer,
                                     max_samples=min(max_samples_per_repeat, X_trans.shape[0]))
    except TypeError:
        res = permutation_importance(est, X_trans, y_arr, n_repeats=n_repeats,
                                     random_state=42, n_jobs=n_jobs, scoring=scorer)

    means = res.importances_mean
    all_imps = res.importances
    imp_df = pd.DataFrame({'feature': feat_names, 'importance': means}).sort_values('importance', ascending=False).reset_index(drop=True)
    imp_df.to_csv(os.path.join(out_dir, f"{model_tag}_perm_importances_mean.csv"), index=False, encoding='utf-8-sig')
    np.save(os.path.join(out_dir, f"{model_tag}_perm_importances_all.npy"), all_imps)

    # Plot
    _plot_top_bar(imp_df, model_tag, top_n)
    _plot_cumulative(imp_df, model_tag, max(50, top_n))
    _plot_perm_uncertainty({'means':means,'all':all_imps,'feature_names':feat_names}, model_tag, top_n)
    return imp_df, {'means':means,'all':all_imps,'feature_names':feat_names}


## 3.1 Classical ML Models


### 3.1.0 Pre-processing for classical ML Models

### 3.1.1 SVM

In [ ]:
# === SVM hyperparameter tuning (Step 1: GridSearch on small subset, safe and efficient) ===
import time
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

# ----- Configuration (if machine resources are limited, reduce dev_sample to 20000 or 10000) -----
dev_sample = 10000  # Number of samples for GridSearch (stratified sampling)
cv_splits = 3       # Number of CV folds (3 is faster, 5 is more stable)
random_state = 42

# ----- Stratified sampling from training set to create dev set and dev_validation -----
if len(X_train) > dev_sample:
    X_dev, _, y_dev, _ = train_test_split(X_train, y_train, train_size=dev_sample, stratify=y_train, random_state=random_state)
else:
    X_dev = X_train.copy()
    y_dev = y_train.copy()

print("Using dev sample size:", len(X_dev), " Positive examples in dev:", int(y_dev.sum()))

# ----- Further split dev into train/val (we also specify StratifiedKFold for GridSearch's internal CV) -----
# Here we directly pass cv=StratifiedKFold(...) to GridSearchCV
cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)

# ----- Build pipeline (preprocessor already constructed in previous step: preprocessor) -----
pipeline_svc = Pipeline([
    ('preproc', preprocessor),
    ('svc', SVC(random_state=random_state))  # Don't set probability=True (saves time), use decision_function for AUC
])

# ----- Parameter grid (keeping your original parameter options) -----
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf'],
    'svc__class_weight': ['balanced', None],
    'svc__gamma': ['scale', 'auto']
}

# ----- GridSearchCV (run in parallel) -----
grid = GridSearchCV(
    estimator=pipeline_svc,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# ----- Time and fit (note: may still take several minutes or longer, depending on dev_sample and kernel) -----
start_time = time.time()
grid.fit(X_dev, y_dev)
end_time = time.time()
print(f"GridSearch done in {end_time - start_time:.1f} seconds")

print("Best params:", grid.best_params_)
print("Best CV score (f1_weighted):", grid.best_score_)

# ----- Get the best estimator (including preprocessor) -----
best_svm_pipeline = grid.best_estimator_

# ----- Save best model for later loading (overwrite old svm_model_binary.pkl) -----
import joblib
joblib.dump(best_svm_pipeline, "svm_model_binary.pkl")
print("Saved best SVM pipeline to svm_model_binary.pkl")


In [ ]:
# 2025.10.17: SVM - importance & plots (force permutation for rbf/nonlinear kernels)
svm_imp_df, svm_perm = run_importance_all_models(
    best_svm_pipeline, X_train, y_train,
    model_tag="SVM",
    top_n=20,
    prefer='always_perm',   # Force permutation (required for SVM rbf)
    n_repeats=10,           # Start with 10; can increase to 20 for final publication
    max_subsample=3000,     # Subsample for speed; can use 5000~10000 on better machines
    n_jobs=1                # Single thread recommended for Windows stability
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    roc_curve,
    auc
)
from sklearn.utils import resample


def bootstrap_f1_ci(y_true, y_pred, n_iterations=1000, average='weighted'):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    f1_scores = []

    for _ in range(n_iterations):
        indices = resample(np.arange(len(y_true)))
        if len(np.unique(y_true[indices])) < 2:
            continue
        f1 = f1_score(y_true[indices], y_pred[indices], average=average)
        f1_scores.append(f1)

    f1_mean = np.mean(f1_scores)
    ci_lower = np.percentile(f1_scores, 2.5)
    ci_upper = np.percentile(f1_scores, 97.5)
    return f1_mean, ci_lower, ci_upper


def bootstrap_auc_ci(y_true, y_scores, n_iterations=1000):
    y_true = np.array(y_true)
    y_scores = np.array(y_scores)
    auc_scores = []

    for _ in range(n_iterations):
        indices = resample(np.arange(len(y_true)))
        if len(np.unique(y_true[indices])) < 2:
            continue
        fpr, tpr, _ = roc_curve(y_true[indices], y_scores[indices])
        auc_score = auc(fpr, tpr)
        auc_scores.append(auc_score)

    ci_lower = np.percentile(auc_scores, 2.5)
    ci_upper = np.percentile(auc_scores, 97.5)
    return np.mean(auc_scores), ci_lower, ci_upper


def plot_confusion_matrix(y_true, y_pred, labels=['0', '1']):
    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


def plot_roc_curve(y_true, y_scores, label_prefix="Model"):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'{label_prefix} AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {label_prefix}')
    plt.legend(loc='lower right')
    plt.show()
    return roc_auc


def evaluate_model(model, X_test, y_test, model_name="Model", use_proba=False):
    print(f"\n--- Evaluation Report: {model_name} ---\n")

    y_pred = model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    f1_mean, f1_ci_low, f1_ci_high = bootstrap_f1_ci(y_test, y_pred)
    print(f"Weighted F1 Score: {f1_mean:.4f}")
    print(f"95% CI for F1 Score: [{f1_ci_low:.4f}, {f1_ci_high:.4f}]")

    plot_confusion_matrix(y_test, y_pred)

    # ROC + AUC
    if use_proba:
        y_scores = model.predict_proba(X_test)[:, 1]
    else:
        y_scores = model.decision_function(X_test)

    auc_score, auc_ci_low, auc_ci_high = bootstrap_auc_ci(y_test, y_scores)
    print(f"AUC Score: {auc_score:.4f}")
    print(f"95% CI for AUC: [{auc_ci_low:.4f}, {auc_ci_high:.4f}]")

    plot_roc_curve(y_test, y_scores, label_prefix=model_name)


# --- Usage Example ---
# For SVM with decision_function:
# evaluate_model(SVM, X_test_1, y_test_1, model_name="SVM", use_proba=False)

# For models like RandomForest or LogisticRegression with predict_proba:
# evaluate_model(rf_model, X_test, y_test, model_name="Random Forest", use_proba=True)


In [ ]:
import joblib

# Load model
loaded_svm = joblib.load("svm_model_binary.pkl")

# Call evaluation function
evaluate_model(loaded_svm, X_test, y_test, model_name="SVM", use_proba=False)

### 3.1.2 Logistic Regression

In [ ]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
# ----- Build pipeline -----
pipeline_lr = Pipeline([
    ('preproc', preprocessor),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
])

# ----- Parameter grid -----
param_grid_lr = {
    'lr__C': [0.01, 0.1, 1, 10, 100],
    'lr__solver': ['liblinear', 'lbfgs'],
    'lr__penalty': ['l2'],
    'lr__class_weight': ['balanced', None]
}

# ----- GridSearchCV -----
grid_lr = GridSearchCV(
    estimator=pipeline_lr,
    param_grid=param_grid_lr,
    scoring='f1_weighted',
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Start timer
start_time = time.time()
grid_lr.fit(X_dev, y_dev)
end_time = time.time()
print(f"Grid search (LogReg) completed in {end_time - start_time:.1f} seconds")

print("Best parameters (LogReg):", grid_lr.best_params_)
print("Best CV score (f1_weighted):", grid_lr.best_score_)

# Save best model
best_lr_pipeline = grid_lr.best_estimator_
joblib.dump(best_lr_pipeline, "lr_model_binary.pkl")
print("Best Logistic Regression pipeline saved to lr_model_binary.pkl")


# Force use of parameters we know are correct
pipeline_lr_fixed = Pipeline([
    ('preproc', preprocessor),
    ('lr', LogisticRegression(
        C=1,  # Use best C value found from grid search
        class_weight='balanced',  # Force balanced weights
        penalty='l2',
        solver='liblinear',
        random_state=42,
        max_iter=1000
    ))
])

# Train on full training set
pipeline_lr_fixed.fit(X_train, y_train)

# Simple evaluation
y_pred = pipeline_lr_fixed.predict(X_test)
print("--- LR-Fixed Results ---")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))



In [ ]:

lr_imp_df, lr_perm = run_importance_all_models(
    best_lr_pipeline, X_train, y_train,
    model_tag="LogisticRegression",
    top_n=20,
    prefer='auto'    # Will automatically use coef_
)


In [ ]:
# Evaluate on test set
evaluate_model(best_lr_pipeline, X_test, y_test, model_name="Logistic Regression", use_proba=True)

### 3.1.3 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# ----- Build pipeline -----
pipeline_rf = Pipeline([
    ('preproc', preprocessor),
    ('rf', RandomForestClassifier(random_state=42, n_jobs=-1))
])

# ----- Parameter grid -----
param_grid_rf = {
    'rf__n_estimators': [50, 100, 200],       # Number of trees
    'rf__max_depth': [10, 20, None],          # Maximum depth of trees
    'rf__min_samples_split': [2, 5, 10],      # Minimum samples to split a node
    'rf__min_samples_leaf': [1, 2, 4],        # Minimum samples at a leaf node
    'rf__class_weight': [None, 'balanced']    # Handle class imbalance
}

# ----- GridSearchCV -----
grid_rf = GridSearchCV(
    estimator=pipeline_rf,
    param_grid=param_grid_rf,
    scoring='f1_weighted',
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Start timer
start_time = time.time()
grid_rf.fit(X_dev, y_dev)
end_time = time.time()
print(f"Grid search (RandomForest) completed in {end_time - start_time:.1f} seconds")

print("Best parameters (RandomForest):", grid_rf.best_params_)
print("Best CV score (f1_weighted):", grid_rf.best_score_)

# Save best model
best_rf_pipeline = grid_rf.best_estimator_
joblib.dump(best_rf_pipeline, "rf_model_binary.pkl")
print("Best RandomForest pipeline saved to rf_model_binary.pkl")

In [ ]:
# 2025.10.17
rf_imp_df, rf_perm = run_importance_all_models(
    best_rf_pipeline, X_train, y_train,
    model_tag="RandomForest",
    top_n=20,
    prefer='auto'    # Will automatically use feature_importances
)

In [ ]:
# Evaluate on test dataset
evaluate_model(best_rf_pipeline, X_test, y_test, model_name="Random Forest", use_proba=True)

### 3.1.4 LGBM

In [ ]:
from lightgbm import LGBMClassifier

# ----- Build pipeline -----
pipeline_lgbm = Pipeline([
    ('preproc', preprocessor),
    ('lgbm', LGBMClassifier(random_state=42, n_jobs=-1, importance_type="gain"))
])

# ----- Parameter grid -----
param_grid_lgbm = {
    'lgbm__n_estimators': [50, 100],
    'lgbm__learning_rate': [0.01, 0.1],
    'lgbm__max_depth': [-1, 10],
    'lgbm__num_leaves': [31, 50],
    'lgbm__min_child_samples': [10, 20],
    'lgbm__class_weight': [None, 'balanced']
}

# ----- GridSearchCV -----
grid_lgbm = GridSearchCV(
    estimator=pipeline_lgbm,
    param_grid=param_grid_lgbm,
    scoring='f1_weighted',
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# ----- Start grid search -----
start_time = time.time()
grid_lgbm.fit(X_dev, y_dev)
end_time = time.time()
print(f"Grid search (LightGBM) completed in {end_time - start_time:.1f} seconds")

print("Best parameters (LightGBM):", grid_lgbm.best_params_)
print("Best CV score (f1_weighted):", grid_lgbm.best_score_)

# ----- Save best model -----
best_lgbm_pipeline = grid_lgbm.best_estimator_
joblib.dump(best_lgbm_pipeline, "lgbm_model_binary.pkl")
print("Best LightGBM pipeline saved to lgbm_model_binary.pkl")

In [ ]:

lgbm_imp_df, lgbm_perm = run_importance_all_models(
    best_lgbm_pipeline, X_train, y_train,
    model_tag="LightGBM",
    top_n=20,
    prefer='auto'    # Will automatically use feature_importances
)

In [ ]:
# evaluate
evaluate_model(best_lgbm_pipeline, X_test, y_test, model_name="LightGBM", use_proba=True)


In [ ]:
# Multi-model comparison heatmap
def build_compare_matrix(model_imp_map, top_k=50, fillna=0.0):
    feats = set()
    for m, df in model_imp_map.items():
        feats |= set(df.head(top_k)['feature'].tolist())
    feats = sorted(list(feats))
    mat = pd.DataFrame(index=feats)
    for m, df in model_imp_map.items():
        s = df.set_index('feature')['importance']
        mat[m] = s.reindex(feats).fillna(fillna)
    return mat

def plot_compare_heatmap(mat_df):
    h = max(4, mat_df.shape[0] * 0.18)
    plt.figure(figsize=(10, h))
    sns.heatmap(mat_df, cmap='viridis', linewidths=0.5)
    plt.title("Model comparison: feature importances")
    plt.tight_layout(); plt.show()

model_imp_map = {}
for name, df in [('LogReg', lr_imp_df), ('RandomForest', rf_imp_df), ('LightGBM', lgbm_imp_df), ('SVM', svm_imp_df)]:
    if df is not None:
        model_imp_map[name] = df

if len(model_imp_map) >= 2:
    mat = build_compare_matrix(model_imp_map, top_k=50)
    plot_compare_heatmap(mat)
else:
    print("Not enough models with importances to build heatmap.")

## 3.2  DL Models


### 3.2.2 GRU

In [ ]:
# === (1) Use fitted parts of preprocessor to build DataLoaders for GRU (direct tabular input) ===
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import joblib

# Hyperparameters (adjust as needed)
BATCH_SIZE_GRU = 256
random_state = 42

# Confirm preprocessor has been fitted
if not hasattr(preprocessor, "transformers_"):
    raise RuntimeError("preprocessor not fitted — please run preprocessor.fit(X_train) first (already done in LGBM code section)")

# 1) Get numeric pipeline (imputer+scaler) and reuse its transform
#    If you want to directly reuse preprocessor.transform(X)[...], but we only need the numeric part's transformation
num_pipeline = preprocessor.named_transformers_.get('num', None)
if num_pipeline is None:
    raise RuntimeError("No 'num' transformer in preprocessor, please check if num_cols is correct")

# 2) Get categories_ from categorical onehot encoder to generate label->index mapping
cat_transformer = preprocessor.named_transformers_.get('cat', None)
if cat_transformer is None:
    raise RuntimeError("No 'cat' transformer in preprocessor, please check if cat_cols is correct")

# onehot step:
if hasattr(cat_transformer, "named_steps") and 'onehot' in cat_transformer.named_steps:
    ohe = cat_transformer.named_steps['onehot']
else:
    raise RuntimeError("onehot step not included in categorical transformer or differently named, please check preprocessor construction")

# categories_ is list of arrays, corresponding to the order of cat_cols
ohe_categories = list(ohe.categories_)  # Each element is all categories for that column (from training set)
# Build cat -> index map for each categorical column, keep unseen mapping to len(categories)
cat_to_index = {}
cat_num_embeddings = {}
for i,c in enumerate(cat_cols):
    cats = list(ohe_categories[i])
    mapping = {str(v): idx for idx, v in enumerate(cats)}  # cat value -> 0..K-1
    cat_to_index[c] = mapping
    cat_num_embeddings[c] = len(cats) + 1  # +1 reserved for unseen

# 3) Dataset class returning input_ids=(num_tensor,cat_tensor), label
class TabularGRUDataset(Dataset):
    def __init__(self, X_df, y_ser, num_cols, cat_cols, num_pipeline, cat_to_index):
        self.X = X_df.reset_index(drop=True).copy()
        self.y = y_ser.reset_index(drop=True).astype(int).copy()
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.num_pipeline = num_pipeline
        self.cat_to_index = cat_to_index

        # numeric transformation using fitted numeric pipeline (imputer+scaler)
        if len(self.num_cols) > 0:
            # num_pipeline.transform accepts DataFrame or array
            self.num_mat = self.num_pipeline.transform(self.X[self.num_cols])
            # ensure float32
            self.num_mat = np.array(self.num_mat, dtype=np.float32)
        else:
            self.num_mat = np.zeros((len(self.X), 0), dtype=np.float32)

        # categorical -> integer indices using cat_to_index mapping
        cat_mats = []
        for c in self.cat_cols:
            vals = self.X[c].fillna("__MISSING__").astype(str).values
            map_c = self.cat_to_index[c]
            int_vals = []
            unseen_idx = len(map_c)   # reserved index for unseen
            for v in vals:
                int_vals.append(map_c.get(v, unseen_idx))
            cat_mats.append(np.array(int_vals, dtype=np.int64))
        if len(cat_mats) > 0:
            self.cat_mat = np.stack(cat_mats, axis=1)
        else:
            self.cat_mat = np.zeros((len(self.X), 0), dtype=np.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        num_row = torch.from_numpy(self.num_mat[idx])    # (n_num,)
        cat_row = torch.from_numpy(self.cat_mat[idx])    # (n_cat,)
        label = torch.tensor(int(self.y.iloc[idx]), dtype=torch.long)
        return {'input_ids': (num_row, cat_row), 'label': label}

# 4) Create datasets/loaders: if X_val not present, split X_train into train+val (consistent with LGBM)
# Make sure X_train, X_test, y_train, y_test exist in workspace
if 'X_val' not in globals():
    # merge labels into X_train copy for splitting convenience
    X_train_for_split = X_train.copy()
    X_train_for_split[label_col] = y_train.values
    X_train_df, X_val_df = train_test_split(X_train_for_split, test_size=0.2, stratify=X_train_for_split[label_col], random_state=random_state)
    # separate y
    y_train_df = X_train_df[label_col]
    X_train_df = X_train_df.drop(columns=[label_col])
    y_val_df = X_val_df[label_col]
    X_val_df = X_val_df.drop(columns=[label_col])
else:
    X_train_df = X_train.copy()
    y_train_df = y_train.copy()
    X_val_df = X_val.copy()
    y_val_df = y_val.copy()

# ensure X_test has label
X_test_df = X_test.copy()
X_test_df[label_col] = y_test.values

# Build dataset objects
train_dataset_3 = TabularGRUDataset(X_train_df, y_train_df, num_cols, cat_cols, num_pipeline, cat_to_index)
val_dataset_3   = TabularGRUDataset(X_val_df, y_val_df, num_cols, cat_cols, num_pipeline, cat_to_index)
test_dataset_3  = TabularGRUDataset(X_test_df.drop(columns=[label_col]), X_test_df[label_col], num_cols, cat_cols, num_pipeline, cat_to_index)

# DataLoaders
train_loader_3 = DataLoader(train_dataset_3, batch_size=BATCH_SIZE_GRU, shuffle=True)
val_loader_3   = DataLoader(val_dataset_3, batch_size=BATCH_SIZE_GRU, shuffle=False)
test_loader_3  = DataLoader(test_dataset_3, batch_size=BATCH_SIZE_GRU, shuffle=False)

# expose cat_num_embeddings dict for model construction
# cat_num_embeddings already prepared above
print("Built GRU loaders. Sizes:", len(train_dataset_3), len(val_dataset_3), len(test_dataset_3))
print("Categorical embedding sizes (per column):")
print({k: cat_num_embeddings[k] for k in list(cat_num_embeddings)[:10]})


In [ ]:


# Define GRU Model
import torch.nn as nn
class GRUSentimentModel(nn.Module):
    def __init__(self, num_cols, cat_cols, cat_num_embeddings, embedding_dim=128, hidden_dim=256, output_dim=2, dropout=0.3):
        super(GRUSentimentModel, self).__init__()
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.n_num = len(num_cols)
        self.n_cat = len(cat_cols)
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        # numeric projection: shared linear (scalar -> embedding_dim)
        self.num_proj = nn.Linear(1, embedding_dim)
        # categorical embeddings: one embedding per categorical feature
        self.cat_embeddings = nn.ModuleDict({
            c: nn.Embedding(cat_num_embeddings[c], embedding_dim) for c in cat_cols
        })
        # GRU and classifier head
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, input_ids):
        # input_ids: tuple (num_tensor, cat_tensor)
        num_tensor, cat_tensor = input_ids
        B = num_tensor.size(0)
        emb_list = []
        # numeric projection -> produce (B, n_num, embedding_dim)
        if self.n_num > 0:
            num_flat = num_tensor.view(-1,1)                    # (B*n_num,1)
            num_proj_flat = self.num_proj(num_flat)             # (B*n_num, emb_dim)
            num_proj = num_proj_flat.view(B, self.n_num, -1)    # (B, n_num, emb_dim)
            emb_list.append(num_proj)
        # categorical embeddings -> (B, n_cat, emb_dim)
        if self.n_cat > 0:
            cat_embs = []
            for i, c in enumerate(self.cat_cols):
                emb_i = self.cat_embeddings[c](cat_tensor[:, i])   # (B, emb_dim)
                cat_embs.append(emb_i.unsqueeze(1))               # (B,1,emb_dim)
            cat_embs = torch.cat(cat_embs, dim=1)
            emb_list.append(cat_embs)
        # concat into sequence (B, seq_len, emb_dim)
        seq = torch.cat(emb_list, dim=1)
        # GRU forward -> use last hidden state
        _, hidden = self.gru(seq)
        out = self.fc(self.dropout(hidden[-1]))
        return out

In [ ]:


import time
from tqdm.auto import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, accuracy_score

# Select device: use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Optional: if using GPU, set random seed for reproducibility (optional)
import random, numpy as np
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(1234)
    # Display GPU name (for confirmation)
    try:
        print("GPU name:", torch.cuda.get_device_name(0))
    except:
        pass

def train_and_evaluate(hyperparams, train_loader, val_loader, vocab_size, output_dim, device):
    embedding_dim = int(hyperparams['embedding_dim'])
    hidden_dim = int(hyperparams['hidden_dim'])
    lr = float(hyperparams['lr'])
    epochs = int(hyperparams.get('epochs', 5))

    # Initialize the model, loss, and optimizer
    model = GRUSentimentModel(
        num_cols=num_cols,
        cat_cols=cat_cols,
        cat_num_embeddings=cat_num_embeddings,
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim
    ).to(device)

    # **MODIFIED**: Compute class weights for the training set
    all_train_labels = []
    for batch in train_loader:
        lbls = batch['label']
        if isinstance(lbls, torch.Tensor):
            lbls = lbls.cpu().numpy()
        else:
            lbls = np.array(lbls)
        all_train_labels.extend(lbls.tolist())

    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(all_train_labels),
        y=all_train_labels
    )
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)  # Convert to PyTorch tensor

    # **MODIFIED**: Use CrossEntropyLoss with class weights
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(hyperparams['epochs']):
        model.train()
        total_loss = 0
        train_loop = tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}", leave=True, position=0)
        for batch in train_loop:
            num_tensor, cat_tensor = batch['input_ids']
            num_tensor = num_tensor.to(device).float()
            cat_tensor = cat_tensor.to(device).long()
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model((num_tensor, cat_tensor))
            loss = criterion(outputs, labels)  # **MODIFIED**: Use CrossEntropyLoss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            train_loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

    # Validation evaluation
    model.eval()
    val_predictions, val_labels = [], []
    val_loop = tqdm(val_loader, desc="Validating", leave=True, position=0)
    with torch.no_grad():
        for batch in val_loop:
            num_tensor, cat_tensor = batch['input_ids']
            num_tensor = num_tensor.to(device).float()
            cat_tensor = cat_tensor.to(device).long()
            labels = batch['label'].to(device)

            outputs = model((num_tensor, cat_tensor))
            preds = torch.argmax(outputs, dim=1)
            val_predictions.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # Calculate F1 score and accuracy
    f1 = f1_score(val_labels, val_predictions, average='weighted')
    accuracy = accuracy_score(val_labels, val_predictions)
    print(f"Validation F-1 Score: {f1:.4f}, Accuracy: {accuracy:.4f}")
    return f1, accuracy, model


def random_hyperparameter_tuning(train_loader, val_loader, vocab_size, output_dim, device, n_iter=20):
    # Define hyperparameter ranges for random search
    param_space = {
        'embedding_dim': (150, 250),
        'hidden_dim': (256, 768),
        'lr': (np.log10(1e-4), np.log10(1e-3)),  # Log-uniform for learning rate
        'epochs': (5, 10),  # **MODIFIED**: Removed `alpha` and `gamma` since FocalLoss is no longer used
    }

    best_f1 = 0
    best_params = None
    best_model = None

    # Random search iterations
    with tqdm(total=n_iter, desc="Random Search Tuning", leave=True, position=0) as pbar:
        for _ in range(n_iter):
            # Sample hyperparameters
            hyperparams = {
                'embedding_dim': np.random.uniform(*param_space['embedding_dim']),
                'hidden_dim': np.random.uniform(*param_space['hidden_dim']),
                'lr': 10**np.random.uniform(*param_space['lr']),
                'epochs': np.random.randint(*param_space['epochs']),
            }
            f1, accuracy, model = train_and_evaluate(hyperparams, train_loader, val_loader, vocab_size, output_dim, device)
            pbar.set_postfix(f1=f1, accuracy=accuracy)
            pbar.update(1)

            if f1 > best_f1:
                best_f1 = f1
                best_params = hyperparams
                best_model = model

    print(f"Best Validation F1 Score: {best_f1:.4f}")
    print(f"Best Hyperparameters: {best_params}")

    # Save the best model
    torch.save(best_model.state_dict(), "best_gru_model.pth")
    return best_params, best_f1, best_model

# Start timer
start_time = time.time()
# Run random search
best_params, best_f1, best_model= random_hyperparameter_tuning(
    train_loader_3,
    val_loader_3,
    vocab_size=None,
    output_dim=2,
    device=device,
    n_iter=10  # Number of random samples to evaluate
)
# End timer
end_time = time.time()
total_time = end_time - start_time
print(f"Total Parameter Tuning Time: {total_time:.2f} seconds")


In [ ]:


# ===========Fixed hyperparameters (random search results)==============
hyperparams = {
    'embedding_dim': 186.88240060019746,
    'hidden_dim': 733.7677322150511,
    'lr': 0.000448103301026688,
    'epochs': 8
}

train_loader = train_loader_3
val_loader=val_loader_3

output_dim=2
device=device
embedding_dim = int(hyperparams['embedding_dim'])
hidden_dim = int(hyperparams['hidden_dim'])
lr = float(hyperparams['lr'])


# Initialize the model, loss, and optimizer
model = GRUSentimentModel(
        num_cols=num_cols,
        cat_cols=cat_cols,
        cat_num_embeddings=cat_num_embeddings,
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim
    ).to(device)

# =====Class weights (addressing severe imbalance)=====
all_train_labels = []
for batch in train_loader:
    all_train_labels.extend(batch['label'].numpy())

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(all_train_labels),
    y=all_train_labels
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)  # Convert to PyTorch tensor

# =====Define loss function and optimizer=====
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# =====Training + validation loop=====
for epoch in range(hyperparams['epochs']):
    model.train()
    total_loss = 0.0
    train_loop = tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{hyperparams['epochs']}", leave=True, position=0)
    for batch in train_loop:
        num_tensor, cat_tensor = batch['input_ids']
        num_tensor = num_tensor.to(device).float()
        cat_tensor = cat_tensor.to(device).long()
        labels = batch['label'].to(device).long()

        optimizer.zero_grad()
        outputs = model((num_tensor, cat_tensor))
        loss = criterion(outputs, labels)  # **MODIFIED**: Use CrossEntropyLoss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        train_loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{hyperparams['epochs']}, Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
model_save_path = "GRU_Binary.pth"
torch.save(model.state_dict(), model_save_path)

In [ ]:
# =========================================================
# =========================================================
# GRU: Permutation Importance (by original columns: num_cols + cat_cols)
# =========================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader

# Helper function: perform inference on DataFrame and return y_pred
@torch.no_grad()
def _gru_predict_on_df(model, X_df, y_ser, num_cols, cat_cols, num_pipeline, cat_to_index,
                       batch_size=512, device="cpu"):
    model.eval()
    ds = TabularGRUDataset(X_df, y_ser, num_cols, cat_cols, num_pipeline, cat_to_index)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False)
    preds = []
    for batch in dl:
        num_t, cat_t = batch['input_ids']
        num_t = num_t.to(device).float()
        cat_t = cat_t.to(device).long()
        logits = model((num_t, cat_t))
        pred = torch.argmax(logits, dim=1)
        preds.extend(pred.cpu().numpy().tolist())
    return np.array(preds, dtype=np.int64)

# Main function: compute GRU permutation importance and generate plots/exports
def run_gru_permutation_importance(model,      # Trained GRU model (already to(device))
                                   X_df, y_ser,# Evaluation data (recommend using validation set X_val_df / y_val_df)
                                   num_cols, cat_cols,
                                   num_pipeline, cat_to_index,
                                   model_tag="GRU",
                                   n_repeats=6,
                                   max_subsample=3000,     # Subsample upper limit for speed
                                   batch_size=512,
                                   device=None,
                                   out_dir="./importance_outputs",
                                   top_n=20):
    """
    Perform permutation for each column (numeric / categorical):
      importance = baseline_f1 - mean(f1_after_permute)
    Feature names use original column names (num_cols + cat_cols)
    """
    assert device is not None, "please pass device (cpu/cuda)"

    # ---- Subsample for speed (consistent with fast version for SVM above) ----
    n_full = len(X_df)
    take = min(max_subsample, n_full)
    if take < n_full:
        # Stratified sampling (maintain y distribution)
        from sklearn.model_selection import StratifiedShuffleSplit
        sss = StratifiedShuffleSplit(n_splits=1, train_size=take, random_state=42)
        idx = next(sss.split(np.zeros(n_full), y_ser))[0]
        X_use = X_df.iloc[idx].reset_index(drop=True).copy()
        y_use = y_ser.iloc[idx].reset_index(drop=True).copy()
        print(f"[{model_tag}] Using subsample: {take}/{n_full}")
    else:
        X_use = X_df.reset_index(drop=True).copy()
        y_use = y_ser.reset_index(drop=True).copy()

    # ---- baseline ----
    base_pred = _gru_predict_on_df(model, X_use, y_use, num_cols, cat_cols, num_pipeline, cat_to_index,
                                   batch_size=batch_size, device=device)
    base_f1 = f1_score(y_use.values, base_pred, average='weighted')
    print(f"[{model_tag}] Baseline F1 (weighted): {base_f1:.4f}")

    # ---- Perform permutation for each feature ----
    feat_names = list(num_cols) + list(cat_cols)
    importances = []
    for col in feat_names:
        scores = []
        for _ in range(n_repeats):
            X_perm = X_use.copy()
            # Shuffle rows within this column (preserve marginal distribution)
            X_perm[col] = np.random.permutation(X_perm[col].values)
            pred_p = _gru_predict_on_df(model, X_perm, y_use, num_cols, cat_cols, num_pipeline, cat_to_index,
                                        batch_size=batch_size, device=device)
            f1_p = f1_score(y_use.values, pred_p, average='weighted')
            scores.append(f1_p)
        imp = max(0.0, base_f1 - float(np.mean(scores)))  # F1 decrease is treated as importance (non-negative)
        importances.append(imp)

    importance_df = pd.DataFrame({'feature': feat_names, 'importance': importances}) \
                     .sort_values('importance', ascending=False) \
                     .reset_index(drop=True)

    # ---- Export CSV ----
    os.makedirs(out_dir, exist_ok=True)
    csv_path = os.path.join(out_dir, f"{model_tag}_perm_importances_mean.csv")
    importance_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"[{model_tag}] Saved permutation importance CSV -> {csv_path}")

    # ---- Main plot: Top 20 horizontal bar ----
    top_df = importance_df.head(top_n).sort_values('importance', ascending=True)
    plt.figure(figsize=(10, 7))
    sns.set_style("whitegrid")
    ax = sns.barplot(x='importance', y='feature', data=top_df, orient='h')
    plt.title(f"{model_tag} - Top {top_n} Feature Importance")
    plt.xlabel("Importance (ΔF1)")
    plt.ylabel("Feature")
    for i, (val, feat) in enumerate(zip(top_df['importance'], top_df['feature'])):
        ax.text(val + 1e-6, i, f"{val:.4f}", va='center')
    plt.tight_layout()
    plt.show()

    # ---- Supplementary plot: Cumulative importance curve ----
    vals = importance_df['importance'].values
    total = vals.sum() if vals.sum() != 0 else 1.0
    cum = np.cumsum(vals) / total * 100
    plt.figure(figsize=(8, 4))
    plt.plot(range(1, len(cum)+1), cum, marker='o')
    plt.xlabel("Number of features included")
    plt.ylabel("Cumulative importance (%)")
    plt.title(f"{model_tag} cumulative importance (%)")
    plt.grid(True)
    plt.axhline(80, color='gray', linestyle='--', linewidth=1)
    plt.tight_layout()
    plt.show()

    return importance_df


gru_imp_df = run_gru_permutation_importance(
    model,
    X_train_df, y_train_df,      # If X_val_df/y_val_df not available
    num_cols, cat_cols,
    num_pipeline, cat_to_index,
    model_tag="GRU",
    n_repeats=6,             # Start with 6 for quick run; can use 10~20 for final publication
    max_subsample=3000,      # Subsample for speed; can use 5000~10000 on better machines
    batch_size=512,
    device=device,           # Keep consistent with training
    out_dir="./importance_outputs",
    top_n=20
)



In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
# Evaluate the best model on the test set
best_model = model
best_model.eval()
test_predictions, test_labels = [], []

# **MODIFIED**: Added probabilities collection for further analysis if needed
test_probabilities = []
test_loop = tqdm(test_loader_3, desc="Testing Best Model", leave=True, position=0)
with torch.no_grad():
    for batch in test_loop:
        num_tensor, cat_tensor = batch['input_ids']
        num_tensor = num_tensor.to(device).float()
        cat_tensor = cat_tensor.to(device).long()
        labels = batch['label'].to(device).long()

        # Forward pass
        outputs = best_model((num_tensor, cat_tensor))

        # **MODIFIED**: Added probability calculation using softmax
        probs = torch.softmax(outputs, dim=1)  # Probabilities for each class
        preds = torch.argmax(probs, dim=1)  # Predicted class

        test_predictions.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

        # **MODIFIED**: Collect probabilities of the positive class (e.g., class 1)
        test_probabilities.extend(probs[:, 1].cpu().numpy())

# Calculate F1 score and accuracy on the test set
test_f1 = f1_score(test_labels, test_predictions, average='weighted')
test_accuracy = accuracy_score(test_labels, test_predictions)

print(f"Test F-1 Score: {test_f1:.4f}, Test Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(test_labels, test_predictions, digits=4))


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_pytorch_model(model, test_loader, device, model_name="GRU", num_classes=2):
    model.eval()
    model.to(device)

    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            # Keep consistent with training: numeric features + categorical features
            num_tensor, cat_tensor = batch['input_ids']
            num_tensor = num_tensor.to(device).float()
            cat_tensor = cat_tensor.to(device).long()
            labels = batch['label'].to(device)

            # Forward pass
            outputs = model((num_tensor, cat_tensor))  # Get logits directly
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    if num_classes == 2:
        positive_probs = all_probs[:, 1]
    else:
        positive_probs = None

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"\n--- Evaluation Report: {model_name} ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"Weighted F1 Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, digits=4))

    # F1 CI
    f1_scores = []
    for _ in range(500):  # bootstrap
        idx = resample(np.arange(len(all_labels)))
        if len(np.unique(all_labels[idx])) < 2:
            continue
        f1_bs = f1_score(all_labels[idx], all_preds[idx], average='weighted')
        f1_scores.append(f1_bs)
    if f1_scores:
        print(f"95% CI for F1 Score: [{np.percentile(f1_scores, 2.5):.4f}, {np.percentile(f1_scores, 97.5):.4f}]")

    # Confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.show()

    # ROC
    if num_classes == 2:
        fpr, tpr, _ = roc_curve(all_labels, positive_probs)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve - {model_name}")
        plt.legend(loc='lower right')
        plt.show()

    return acc, f1


gru = GRUSentimentModel(
    num_cols=num_cols,
    cat_cols=cat_cols,
    cat_num_embeddings=cat_num_embeddings,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim
).to(device)
gru.load_state_dict(torch.load("GRU_Binary.pth", map_location=device))
gru.to(device)
evaluate_pytorch_model(gru, test_loader_3, device, model_name="gru", num_classes=2)
